<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model


In [1]:
# !pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-37/checkpoint-32000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-37/checkpoint-32000")

loading configuration file /content/Model/Models-Train-37/checkpoint-32000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "@accessori_saldatore",
    "1": "@accessori_trapano",
    "2": "@armadio_cassettiera",
    "3": "@bancale_cargo",
    "4": "@banco",
    "5": "@brugola",
    "6": "@cacciavite_giravite_cercafase",
    "7": "@carrello",
    "8": "@carta-vetrata_carta-abrasiva",
    "9": "@cassaforte",
    "10": "@catena",
    "11": "@cazzuola_frattone",
    "12": "@cerniera_bandella",
    "13": "@chiave-a-bussola_tubo",
    "14": "@chiave_inglese_poligonale",
    "15": "@chiavistello",
    "16": "@chiodo_rivetto",
    "17": "@colla_silicone",
    "18": "@collare",
    "19": "@coltello",
    "20": "@estrattore",
    "21": "@fo

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/jointHF-train+test_unseen/test", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
# for i in range(0,10):
#     sample = ds[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [ ]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-ferramenta-val", split="train")

In [ ]:
ds_f

In [ ]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.9

In [ ]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

In [ ]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

In [ ]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [ ]:
# selected_indices

In [ ]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [ ]:
# selected_indices

In [ ]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [ ]:
fused_ds

In [ ]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-ferramenta/images-val", split="train")

In [ ]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [ ]:
ds_imgs = ds_joint.filter(filter_funtion)

In [ ]:
ds_imgs

In [ ]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [ ]:
all_indices = range(len(ds_imgs))
print(all_indices)

In [ ]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [ ]:
# remaining_indices

In [ ]:
missing_ds = ds_imgs.select(remaining_indices)

In [ ]:
missing_ds

In [ ]:
# Combine Both Datasets

In [ ]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [ ]:
ds

# Pre-Processing and Final Validation Loop

In [ ]:
labels = ds.features["label"]
labels

In [ ]:
labels.int2str(ds[532]["label"])

In [ ]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [ ]:
dataset = ds.with_transform(transform)

In [ ]:
dataset

In [ ]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

In [ ]:
labels = ds.features["label"].names
labels

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
dataset

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

In [ ]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
model = model.to(device)
model.eval()
predictions, labels = [], []
# valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    # loss = outputs.loss
    # valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [ ]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
print("AUROC_score: ", AUROC_score)

In [ ]:
print(classification_report(labels, predictions))

In [ ]:
cm = confusion_matrix(labels, predictions)
print(cm)

# Joint Representation Experiment


In [4]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint_ferramenta/mulitmodal_img_enc_txt/images-val", split="train")

Resolving data files:   0%|          | 0/43738 [00:00<?, ?it/s]

In [5]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 43738
})

In [6]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [7]:
ds_3 = ds.filter(filter_funtion)

In [8]:
ds_3

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [9]:
for i in range(0,5):
    sample = ds_3[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

File Name: 123031564.jpg_3.png
File Name: 125066170.jpg_3.png
File Name: 128436452.jpg_3.png
File Name: 129979053.jpg_3.png
File Name: 130236413.jpg_3.png


Image Dataset

In [10]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [11]:
ds_4 = ds.filter(filter_funtion)

In [12]:
ds_4

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [13]:
for i in range(0,5):
    sample = ds_4[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

File Name: 123031564.jpg_4.png
File Name: 125066170.jpg_4.png
File Name: 128436452.jpg_4.png
File Name: 129979053.jpg_4.png
File Name: 130236413.jpg_4.png


Pre-processing

In [14]:
labels = ds.features["label"]
labels

ClassLabel(names=['@accessori_saldatore', '@accessori_trapano', '@armadio_cassettiera', '@bancale_cargo', '@banco', '@brugola', '@cacciavite_giravite_cercafase', '@carrello', '@carta-vetrata_carta-abrasiva', '@cassaforte', '@catena', '@cazzuola_frattone', '@cerniera_bandella', '@chiave-a-bussola_tubo', '@chiave_inglese_poligonale', '@chiavistello', '@chiodo_rivetto', '@colla_silicone', '@collare', '@coltello', '@estrattore', '@forbice_cesoia', '@lama_seghetto_sega', '@lame_sega_circolare', '@lima_grattuggia_raspa', '@livella', '@lucchetto', '@maniglia_pomello', '@martello_mazzetta_ascia', '@metro_flessometro', '@morsetto_strettoio', '@nastro', '@pennello', '@pinza_punzonatrice_giratubo', '@pinzetta', '@pistola', '@rubinetto_miscelatore_valvola', '@ruota', '@scaffale', '@scala', '@scalpello', '@sega-a-tazza', '@seghetto_sega', '@segnaletica', '@serratura_cilindro', '@spatola_raschietto', '@spazzola', '@staffa_squadretta', '@tassello', '@torcia-faro', '@vernice', '@vite_bullone'], id=Non

In [15]:
labels.int2str(ds[532]["label"])

'@accessori_trapano'

In [16]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [17]:
dataset_3 = ds_3.with_transform(transform)
dataset_4 = ds_4.with_transform(transform)

In [18]:
dataset_3

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [19]:
dataset_4

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [20]:
for item in dataset_4:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [21]:
labels = ds.features["label"].names
labels

['@accessori_saldatore',
 '@accessori_trapano',
 '@armadio_cassettiera',
 '@bancale_cargo',
 '@banco',
 '@brugola',
 '@cacciavite_giravite_cercafase',
 '@carrello',
 '@carta-vetrata_carta-abrasiva',
 '@cassaforte',
 '@catena',
 '@cazzuola_frattone',
 '@cerniera_bandella',
 '@chiave-a-bussola_tubo',
 '@chiave_inglese_poligonale',
 '@chiavistello',
 '@chiodo_rivetto',
 '@colla_silicone',
 '@collare',
 '@coltello',
 '@estrattore',
 '@forbice_cesoia',
 '@lama_seghetto_sega',
 '@lame_sega_circolare',
 '@lima_grattuggia_raspa',
 '@livella',
 '@lucchetto',
 '@maniglia_pomello',
 '@martello_mazzetta_ascia',
 '@metro_flessometro',
 '@morsetto_strettoio',
 '@nastro',
 '@pennello',
 '@pinza_punzonatrice_giratubo',
 '@pinzetta',
 '@pistola',
 '@rubinetto_miscelatore_valvola',
 '@ruota',
 '@scaffale',
 '@scala',
 '@scalpello',
 '@sega-a-tazza',
 '@seghetto_sega',
 '@segnaletica',
 '@serratura_cilindro',
 '@spatola_raschietto',
 '@spazzola',
 '@staffa_squadretta',
 '@tassello',
 '@torcia-faro',
 '@v

In [22]:
num_of_classes = len(labels)
print(num_of_classes)

52


In [23]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [24]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [25]:
batch_size = 32

In [26]:
valid_dataset_loader_4 = DataLoader(dataset_4, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [27]:
print(len(valid_dataset_loader_4))

684


In [28]:
valid_dataset_loader_3 = DataLoader(dataset_3, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [29]:
print(len(valid_dataset_loader_3))

684


In [30]:
num_batches = len(valid_dataset_loader_3)
num_batches_with_logits_3 = int(num_batches * 0.3)  # ___% of batches that you want to include (30 for now)
print("Total Batches: ", num_batches)
print("Missing Modaility Batches: ", num_batches_with_logits_3)

Total Batches:  684
Missing Modaility Batches:  205


In [31]:
import random

controller = random.sample(range(1, num_batches+1), num_batches_with_logits_3)
controller = sorted(controller)

In [32]:
len(controller)

205

Validation Loops

In [33]:
model = model.to(device)
model.eval()
predictions_4, labels_4 = [], []
for batch in tqdm(valid_dataset_loader_3):
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    logits = outputs.logits.detach().cpu()
    probabilities = torch.softmax(logits, dim=1)

    predictions_4.extend(probabilities.cpu().numpy())
    labels_4.extend(label_ids.cpu().numpy())

  0%|          | 0/684 [00:00<?, ?it/s]

In [34]:
predictions_3 = []
batch_idx = 0
for batch in tqdm(valid_dataset_loader_4):
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    if batch_idx in controller:
        # print(batch_idx)
        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()
    else:
        length = len(label_ids)
        logits = torch.zeros((length, num_of_classes), device="cpu")

    probabilities = torch.softmax(logits, dim=1)
    predictions_3.extend(probabilities.cpu().numpy())
    batch_idx = batch_idx + 1

  0%|          | 0/684 [00:00<?, ?it/s]

In [35]:
import numpy as np
predictions_3 = np.array(predictions_3)
predictions_4 = np.array(predictions_4)

In [36]:
predictions_avg = (predictions_3 + predictions_4) / 2
predictions = np.argmax(predictions_avg, axis=1)

In [37]:
predictions = np.array(predictions)
labels = np.array(labels_4)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [39]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.9328272897709086
precision:  0.933241746801171
f1_score:  0.9327347735755003
recall 0.9328272897709086


In [40]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.73      0.54      0.62        59
           1       0.96      0.97      0.96      1879
           2       0.97      0.94      0.95       345
           3       0.98      1.00      0.99        48
           4       0.92      0.98      0.95       132
           5       0.91      0.86      0.88       176
           6       0.97      0.99      0.98      1069
           7       0.95      0.92      0.93       148
           8       0.99      0.96      0.97       663
           9       0.97      0.99      0.98       642
          10       0.90      0.91      0.91       136
          11       0.80      0.91      0.85        35
          12       0.91      0.88      0.89       112
          13       0.92      0.93      0.92       842
          14       0.95      0.96      0.96      1555
          15       0.71      0.69      0.70        36
          16       1.00      0.67      0.80        51
          17       0.92    

In [41]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[  32    2    0 ...    2    0    0]
 [   0 1818    0 ...    0    0    2]
 [   0    0  325 ...    0    0    0]
 ...
 [   0    0    0 ...  151    0    0]
 [   1    0    0 ...    0  102    0]
 [   0    3    0 ...    0    0 2166]]
